# UCK 358E – Introduction to Artificial Intelligence 
## Final Exam – Part 2
## June 11, 2023




#### Name: 
#### Student Number:

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

## import any library that you will use


In [9]:
# Functions for loading and plotting
def loadData():
    df = pd.read_csv("/kaggle/input/uck358data/dataAll.txt", sep="\t", header=None, names=["f"+str(i) for i in range(1,66)])
    df.rename(columns = {'f65':'Class'}, inplace = True)
    return df

def split2DataSets(df_x, df_y):
    # don't change anything in this function
    strtfdSplit = StratifiedShuffleSplit(n_splits=10, test_size=0.95, random_state=32)
    kfold = strtfdSplit.split(df_x, df_y)
    dataSets = []
    for k, (train_idx, test_idx) in enumerate(kfold):
        dataSets.append({"X_train": df_x.iloc[train_idx, :], "y_train": df_y.iloc[train_idx],
                        "X_test":df_x.iloc[test_idx, :], "y_test": df_y.iloc[test_idx]})
    return dataSets

def obtainSubDataSet(dSetId):
    X_train = dataSets[dSetId]["X_train"]; y_train = dataSets[dSetId]["y_train"]
    X_test = dataSets[dSetId]["X_test"]; y_test = dataSets[dSetId]["y_test"]
    return X_train, y_train, X_test, y_test    

def plotDecisionBoundary(model, X, y):
    x_min, x_max = X[0].min() - 1, X[0].max() + 1
    y_min, y_max = X[1].min() - 1, X[1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                         np.arange(y_min, y_max, 0.1))
    X_plot = np.c_[xx.ravel(), yy.ravel()]
    y_plot = model.decision_function(X_plot).reshape(xx.shape)
    
    plt.figure()
    plt.scatter(X[np.array(y == 1)][0], X[np.array(y == 1)][1], color="red")
    plt.scatter(X[np.array(y == 0)][0], X[np.array(y == 0)][1], color="blue")
    plt.contour(xx, yy, y_plot, levels=[0])
    plt.show()

In [11]:
loadData()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f56,f57,f58,f59,f60,f61,f62,f63,f64,Class
0,0,0,5,13,9,1,0,0,0,0,...,0,0,0,6,13,10,0,0,0,0
1,0,0,1,9,15,11,0,0,0,0,...,0,0,0,1,10,13,3,0,0,0
2,0,0,3,13,11,7,0,0,0,0,...,0,0,0,2,12,13,4,0,0,0
3,0,0,10,14,11,3,0,0,0,4,...,0,0,0,11,16,12,3,0,0,0
4,0,0,6,14,10,2,0,0,0,0,...,0,0,0,7,16,11,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0,0,4,10,12,7,0,0,0,0,...,0,0,0,1,9,10,0,0,0,1
356,0,0,4,16,16,8,0,0,0,0,...,0,0,0,6,12,12,2,0,0,1
357,0,0,1,12,10,3,0,0,0,0,...,0,0,0,1,7,12,10,0,0,1
358,0,0,8,12,11,6,0,0,0,0,...,0,0,0,4,11,12,7,0,0,1


## Load and analyze whole data for feature selection

### 1) Remove at least 10 features without using class/label information (don't remove more than 20)

In [12]:
df = loadData()

In [13]:
df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f56,f57,f58,f59,f60,f61,f62,f63,f64,Class
0,0,0,5,13,9,1,0,0,0,0,...,0,0,0,6,13,10,0,0,0,0
1,0,0,1,9,15,11,0,0,0,0,...,0,0,0,1,10,13,3,0,0,0
2,0,0,3,13,11,7,0,0,0,0,...,0,0,0,2,12,13,4,0,0,0
3,0,0,10,14,11,3,0,0,0,4,...,0,0,0,11,16,12,3,0,0,0
4,0,0,6,14,10,2,0,0,0,0,...,0,0,0,7,16,11,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0,0,4,10,12,7,0,0,0,0,...,0,0,0,1,9,10,0,0,0,1
356,0,0,4,16,16,8,0,0,0,0,...,0,0,0,6,12,12,2,0,0,1
357,0,0,1,12,10,3,0,0,0,0,...,0,0,0,1,7,12,10,0,0,1
358,0,0,8,12,11,6,0,0,0,0,...,0,0,0,4,11,12,7,0,0,1


In [38]:
drop_indices = np.random.choice(df.columns, 10, replace=False)

df_removed = df.drop(drop_indices,axis=1)
df_removed

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f53,f54,f55,f56,f57,f59,f60,f62,f64,Class
0,0,0,5,13,9,1,0,0,0,0,...,10,12,0,0,0,6,13,0,0,0
1,0,0,1,9,15,11,0,0,0,0,...,10,10,0,0,0,1,10,3,0,0
2,0,0,3,13,11,7,0,0,0,0,...,9,14,1,0,0,2,12,4,0,0
3,0,0,10,14,11,3,0,0,0,4,...,11,15,0,0,0,11,16,3,0,0
4,0,0,6,14,10,2,0,0,0,0,...,14,10,0,0,0,7,16,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0,0,4,10,12,7,0,0,0,0,...,16,12,0,0,0,1,9,0,0,1
356,0,0,4,16,16,8,0,0,0,0,...,15,2,0,0,0,6,12,2,0,1
357,0,0,1,12,10,3,0,0,0,0,...,16,8,0,0,0,1,7,10,0,1
358,0,0,8,12,11,6,0,0,0,0,...,16,8,0,0,0,4,11,7,0,1


,Class
0,0
1,0
2,0
3,0
4,0
...,...
355,1
356,1
357,1
358,1


In [39]:
df_x = df_removed.loc[:, df_removed.columns != 'Class']
df_y = df_removed.loc[:, df_removed.columns == 'Class']
# You will use the new data obtained after removing several features for the rest of the study 

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f52,f53,f54,f55,f56,f57,f59,f60,f62,f64
0,0,0,5,13,9,1,0,0,0,0,...,5,10,12,0,0,0,6,13,0,0
1,0,0,1,9,15,11,0,0,0,0,...,12,10,10,0,0,0,1,10,3,0
2,0,0,3,13,11,7,0,0,0,0,...,14,9,14,1,0,0,2,12,4,0
3,0,0,10,14,11,3,0,0,0,4,...,1,11,15,0,0,0,11,16,3,0
4,0,0,6,14,10,2,0,0,0,0,...,11,14,10,0,0,0,7,16,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0,0,4,10,12,7,0,0,0,0,...,16,16,12,0,0,0,1,9,0,0
356,0,0,4,16,16,8,0,0,0,0,...,16,15,2,0,0,0,6,12,2,0
357,0,0,1,12,10,3,0,0,0,0,...,16,16,8,0,0,0,1,7,10,0
358,0,0,8,12,11,6,0,0,0,0,...,16,16,8,0,0,0,4,11,7,0


In [41]:
X_train,Y_train,x_test,y_test = split2DataSets(df_x, df_y)

ValueError: too many values to unpack (expected 4)

## Model Training

In [44]:
# Data is splitted into 10 subdatasets. "dataSets" contains these 10 different subsets
# Assume that these subsets are indepent from each other. During the training and testing, 
# only use the data that belongs to the focused subset

dataSets = split2DataSets(df_x, df_y)

In [50]:
# for example, set 2 contains it's own train and test data. 
# You can use "obtainSubDataSet" function to get train and test data for any subdataset, use its ID (between 0-9) to get data
X_train, y_train, X_test, y_test = obtainSubDataSet(9)

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f52,f53,f54,f55,f56,f57,f59,f60,f62,f64
143,0,0,2,12,7,0,0,0,0,0,...,12,4,14,3,0,0,1,13,9,0
77,0,0,2,12,15,3,0,0,0,0,...,16,14,15,0,0,0,0,10,4,0
338,0,0,0,13,14,0,0,0,0,0,...,16,7,0,0,0,0,1,11,0,0
257,0,0,4,12,13,3,0,0,0,0,...,13,16,3,0,0,0,0,15,11,0
96,0,0,8,14,11,2,0,0,0,6,...,3,4,15,3,0,0,5,16,7,0
246,0,0,0,14,16,15,11,0,0,0,...,16,15,2,0,0,0,1,15,1,0
26,0,0,4,11,15,8,0,0,0,0,...,12,14,10,0,0,0,4,14,1,0
270,0,0,16,8,0,0,0,0,0,2,...,12,16,11,8,3,0,12,16,16,9
135,0,0,1,12,12,3,0,0,0,0,...,6,8,9,0,0,0,2,14,2,0
218,0,0,1,8,16,7,0,0,0,0,...,11,16,16,1,0,0,0,5,7,0


### 2) Classify each subdataset independently using SVM with linear, sigmoid,  polynomial and radial basis function kernel. Give the classification accuracies on training and test sets (as a table). 

In [10]:
# don't mix the different datasets, but you can use the results of different datasets for hyperparameter tuning

In [54]:
from sklearn.utils import validation
y_train=validation.column_or_1d(y_train)

In [79]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')


In [80]:
svm.fit(X_train,y_train)

SVC(kernel='linear')

In [81]:
 print("print accuracy of svm algo: ",svm.score(X_test,y_test))

print accuracy of svm algo:  1.0


In [86]:
svm = SVC(kernel='linear')
svm.fit(X_train,y_train)

SVC(kernel='linear')

In [87]:
 print("print accuracy of svm algo: ",svm.score(X_test,y_test))

print accuracy of svm algo:  1.0


### 3) Use all samples in the training sets of 10 subsets to train a Neural Network (Classifier) in which there is only one hidden layer with two neurons (assume the activation function in hidden layer is linear). Reduce the dimensionality of the dataset using the outputs of hidden units (Output of the hidden units for each example will be your new feature vector for that example). Plot the reduced dataset in two dimensional spaces (Hidden1 vs Hidden2) using the test samples and illustrating the samples in different classes with different colors. 

In [94]:
from sklearn.neural_network import MLPClassifier

In [56]:
mlp= MLPClassifier(activation ="identity",hidden_layer_sizes=(2),solver='adam',max_iter=1000)
mlp

MLPClassifier(activation='identity', hidden_layer_sizes=2, max_iter=1000)

In [57]:
mlp.fit(X_train,y_train)

MLPClassifier(activation='identity', hidden_layer_sizes=2, max_iter=1000)

In [66]:
from sklearn.decomposition import PCA
hidden_outputs = mlp.predict(X_test)

hidden_outputs


array([1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [67]:
n_components = hidden_outputs
if n_components > 1:
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(X_train)
else:
    reduced_features = X_train

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [64]:
from sklearn.metrics import classification_report, confusion_matrix
pred=mlp.predict(X_test)
confusion_matrix(y_test,pred)

array([[136,  33],
       [ 16, 157]])

In [108]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96       169
           1       1.00      0.92      0.96       173

    accuracy                           0.96       342
   macro avg       0.96      0.96      0.96       342
weighted avg       0.96      0.96      0.96       342



### 4) Use the feature transformation obtained with NN (in Q3) to train SVM classifiers. Classify each subdataset independently using SVM with linear, sigmoid,  polynomial and radial basis function kernel. Give the classification accuracies on training and test sets (as a table). This is similar to Q2, but you have two-dimensional input space (Hidden1 vs Hidden2) for SVM training now. For each case, plot also the decision boundaries using the "plotDecisionBoundary" function.   

### 5) Repeat the Q3 by training an Autoencoder instead of a Neural Network Classifier. The number of nodes in the code layer must be 2 to obtain 2 new features as in Q3. Plot the reduced dataset in two dimensional spaces (Hidden1 vs Hidden2) using the test samples and illustrating the samples in different classes with different colors. 